<a href="https://colab.research.google.com/github/Usool-Data-Science/Computer-Vision-notebooks/blob/main/Feature_Extraction_Stage-of-imageFusion-Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image
import os
from google.colab import files

In [5]:
%cd /content/drive/MyDrive
%mkdir imageFusion
%cd imageFusion

/content/drive/MyDrive
/content/drive/MyDrive/imageFusion


In [6]:
upload = files.upload()

Saving lytro-01-A.jpg to lytro-01-A.jpg
Saving lytro-01-B.jpg to lytro-01-B.jpg
Saving lytro-02-A.jpg to lytro-02-A.jpg
Saving lytro-02-B.jpg to lytro-02-B.jpg
Saving lytro-03-A.jpg to lytro-03-A.jpg
Saving lytro-03-B.jpg to lytro-03-B.jpg
Saving lytro-04-A.jpg to lytro-04-A.jpg
Saving lytro-04-B.jpg to lytro-04-B.jpg
Saving lytro-05-A.jpg to lytro-05-A.jpg
Saving lytro-05-B.jpg to lytro-05-B.jpg
Saving lytro-06-A.jpg to lytro-06-A.jpg
Saving lytro-06-B.jpg to lytro-06-B.jpg
Saving lytro-07-A.jpg to lytro-07-A.jpg
Saving lytro-07-B.jpg to lytro-07-B.jpg
Saving lytro-08-A.jpg to lytro-08-A.jpg
Saving lytro-08-B.jpg to lytro-08-B.jpg
Saving lytro-09-A.jpg to lytro-09-A.jpg
Saving lytro-09-B.jpg to lytro-09-B.jpg
Saving lytro-10-A.jpg to lytro-10-A.jpg
Saving lytro-10-B.jpg to lytro-10-B.jpg
Saving lytro-11-A.jpg to lytro-11-A.jpg
Saving lytro-11-B.jpg to lytro-11-B.jpg
Saving lytro-12-A.jpg to lytro-12-A.jpg
Saving lytro-12-B.jpg to lytro-12-B.jpg
Saving lytro-13-A.jpg to lytro-13-A.jpg


In [8]:
# Define the convolutional layer
conv_layer = nn.Sequential(
    nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1),
    nn.BatchNorm2d(16),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
    nn.BatchNorm2d(32),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2)
)


In [9]:
# Define the self-attention layer
class SelfAttention(nn.Module):
    def __init__(self, in_channels):
        super(SelfAttention, self).__init__()

        self.query_conv = nn.Conv2d(in_channels, in_channels // 8, kernel_size=1)
        self.key_conv = nn.Conv2d(in_channels, in_channels // 8, kernel_size=1)
        self.value_conv = nn.Conv2d(in_channels, in_channels, kernel_size=1)
        self.gamma = nn.Parameter(torch.zeros(1))

        self.softmax = nn.Softmax(dim=-1)

    def forward(self, x):
        batch_size, channels, height, width = x.size()
        
        # Project the inputs to query, key, and value
        proj_query = self.query_conv(x).view(batch_size, -1, width * height).permute(0, 2, 1)
        proj_key = self.key_conv(x).view(batch_size, -1, width * height)

        # Compute the attention scores
        energy = torch.bmm(proj_query, proj_key)
        attention = self.softmax(energy)

        # Apply the attention to the value
        proj_value = self.value_conv(x).view(batch_size, -1, width * height)
        out = torch.bmm(proj_value, attention.permute(0, 2, 1))
        out = out.view(batch_size, channels, height, width)

        # Apply the scaling factor and add to the input
        out = self.gamma * out + x

        return out

In [10]:
# Define the image transformer
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [7]:
%cd /content/drive/MyDrive/imageFusion
%mkdir FusionOutput

/content/drive/MyDrive/imageFusion


In [15]:
# Instantiate the self-attention layer
self_att_layer = SelfAttention(32)

# Define the directory path and loop through the images
directory_path = '/content/drive/MyDrive/imageFusion'
output_path = '/content/drive/MyDrive/imageFusion/FusionOutput'
for filename in os.listdir(directory_path):
    if filename.endswith('.jpg') or filename.endswith('.jpeg') or filename.endswith('.png'):
        # Load the image
        image = Image.open(os.path.join(directory_path, filename))
        # Transform the image
        image = transform(image).unsqueeze(0)
        # Extract the features using the convolutional layer
        features = conv_layer(image)
        # Apply the self-attention layer to the features
        spatial_features = self_att_layer(features)
        # Save the spatial features to disk
        output_filename = os.path.splitext(filename)[0] + '.pt'
        torch.save(spatial_features, os.path.join(output_path, output_filename))


We have saved the output as a .pt file which we can load into pictorial feature maps using the cell below

In [ ]:
import matplotlib.pyplot as plt

# Load the feature map for the first input image
#feature_map = torch.load('/path/to/first_image.pt')
feature_map = torch.load('/content/drive/MyDrive/imageFusion/FusionOutput/lytro-01-A.pt')

# Select the first channel of the feature map
channel = 0
channel_map = feature_map[0, channel, :, :]

# Plot the feature map
plt.imshow(channel_map.detach().numpy())
plt.show()
